In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization
from hloc import colmap_for_habitat, triangulation, localize_sfm, visualization
import h5py
import os
import numpy as np
from collections import defaultdict
import pycolmap
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import math
import g2o
from enum import Enum
import cv2
from threading import Lock
from hloc.utils.parsers import (
    parse_image_lists_with_intrinsics, parse_retrieval, names_to_pair)
from tqdm import tqdm
import json

[06/08/2021 10:22:05 INFO] Generating new fontManager, this may take some time...


## Setup
In this notebook, we will run SfM reconstruction from scratch on a set of images. We choose the [South-Building dataset](https://openaccess.thecvf.com/content_cvpr_2013/html/Hane_Joint_3D_Scene_2013_CVPR_paper.html) - we will download it later. First, we define some paths.

In [2]:
dataset = Path('/datasets/Habitat/')
images_path = dataset / '1LXtFkjw3qL_point0/images'

outputs = Path('/datasets/Habitat/Hierarchical_Localization_outputs/sfm/')
# sfm_pairs = outputs / 'pairs_exhaustive_base-around.txt'
# sfm_pairs = outputs / 'pairs_exhaustive.txt'
sfm_dir = outputs / 'sfm_superpoint+superglue'
sfm_pairs = dataset / 'NetVLAD/NetVLAD_database_top_40.txt' 
loc_pairs = dataset / 'NetVLAD/NetVLAD_query_top_40.txt'  # top 50 retrieved by NetVLAD

feature_conf = extract_features.confs['superpoint_habitat']
matcher_conf = match_features.confs['superglue']

features = feature_conf['output']
feature_filename = f"{features}.h5"
match_filename = f"{features}_{matcher_conf['output']}_{sfm_pairs.stem}.h5"
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build

results = outputs / 'Habitat_hloc_superpoint+superglue_netvlad40.txt'  # the result file

In [3]:
cx, cy = 128, 128
fx, fy = 128, 128
width, height = 256., 256.
fx_baseline = 300

## Extract local features

In [4]:
outputs

PosixPath('/datasets/Habitat/Hierarchical_Localization_outputs/sfm')

In [ ]:
ls /datasets/Habitat/Hierarchical_Localization_outputs/sfm

In [ ]:
extract_features.main(feature_conf, images_path, outputs)

## Matching pairs in database with SuperGlue

In [ ]:
match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

## Triangulate a new SfM model from the given poses
We triangulate the sparse 3D pointcloud given the matches and the reference poses. These are obtained from the NVM SIFT model by generating a new COLMAP model without 3D points.

In [ ]:
colmap_for_habitat.main(
    dataset / '1LXtFkjw3qL_point0/images',
    dataset / 'COLMAP/data.txt',
    outputs / 'sfm_empty',
    skip_points=True)

triangulation.main(
    reference_sfm,
    outputs / 'sfm_empty',
    images,
    sfm_pairs,
    outputs / f"{feature_conf['output']}.h5",
    outputs / f"{feature_conf['output']}_{matcher_conf['output']}_{sfm_pairs.stem}.h5",
    colmap_path='colmap')  # change if COLMAP is not in your PATH

## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [ ]:
match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. The file `Aachen_hloc_superpoint+superglue_netvlad50.txt` will contain the estimated query poses. Have a look at `Aachen_hloc_superpoint+superglue_netvlad50.txt_logs.pkl` to analyze some statistics and find failure cases.

In [ ]:
localize_sfm.main(
    reference_sfm / 'model',
    dataset / 'query_with_intrinsics.txt',
    loc_pairs,
    outputs / f"{feature_conf['output']}.h5",
    outputs / f"{feature_conf['output']}_{matcher_conf['output']}_{loc_pairs.stem}.h5",
    results,
    covisibility_clustering=False)  # not required with SuperPoint+SuperGlue

## Visualizing the localization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [ ]:
visualization.visualize_loc(
    results, images, reference_sfm / 'model', n=100, top_k_db=1, prefix='', seed=2)

## Visualization

In [ ]:
f = sfm_pairs.open()
lines = f.readlines()
filename1 = images / Path(lines[0].split(' ')[0])
filename2 = images / Path(lines[1].split(' ')[1].rstrip('\n'))

### Visualization of matched keypoints without RANSAC

In [ ]:
f = sfm_pairs.open()
lines = f.readlines()
filename1 = images / Path(lines[0].split(' ')[0])
filename2 = images / Path(lines[1].split(' ')[1].rstrip('\n'))

In [ ]:
for i in range(10):
    line = lines[i]
    print(line)
    filename1 = images / Path(line.split(' ')[0])
    filename2 = images / Path(line.split(' ')[1].rstrip('\n'))
    visualization.visualize_matches(outputs / feature_file, 
                                outputs / match_file, 
                                images, 
                                filename1, filename2, dpi=100)

### Visualization using SfM model of the map
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [ ]:
visualization.visualize_sfm_2d(sfm_dir / 'models/0', images, color_by='visibility', n=5)

In [ ]:
visualization.visualize_sfm_2d(sfm_dir / 'models/0', images, color_by='track_length', n=5)

In [ ]:
visualization.visualize_sfm_2d(sfm_dir / 'models/0', images, color_by='depth', n=5)

# Implementing optimizationon selected images

In [4]:
cx, cy = 128, 128
fx, fy = 128, 128
width, height = 256., 256.
ransac_thresh = 12.0

In [5]:
def quaternion_to_rotation_matrix(qvec):
    r = R.from_quat([qvec[1], qvec[2], qvec[3], qvec[0]])
    result = r.as_matrix()
    result[:3,2] = -result[:3,2]
    result[:3,1] = -result[:3,1]
    return result

In [6]:
def get_point_3d(x, y, depth, fx, fy, cx, cy, cam_center_world, R_world_to_cam, w_in_quat_first = True):
    if depth <= 0:
        return 0
    new_x = (x - cx)*depth/fx
    new_y = (y - cy)*depth/fy
    new_z = depth
    coord_3D_world_to_cam = np.array([new_x, new_y, new_z], float)
    if len(R_world_to_cam) == 4:
        if w_in_quat_first:
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
        else:
            R_world_to_cam = [R_world_to_cam[3], R_world_to_cam[0], R_world_to_cam[1], R_world_to_cam[2]]
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
    elif len(R_world_to_cam) == 3:
        matrix = R_world_to_cam
    coord_3D_cam_to_world = np.matmul(matrix, coord_3D_world_to_cam) + cam_center_world
    return coord_3D_cam_to_world

In [7]:
def names_to_pair(name0, name1):
    return '_'.join((name0.replace('/', '-'), name1.replace('/', '-')))

In [8]:
def yaw_pitch_roll_from_quaternion(quaternion, w_first=False):
    if not(w_first):
        r = R.from_quat(quaternion)
    else:
        r = R.from_quat([quaternion[1], quaternion[2], quaternion[3] ,quaternion[0]])
    return r.as_euler('yxz', degrees=True)

In [9]:
def yaw_pitch_roll_from_matrix(matrix):
    r = R.from_matrix(matrix)
    return(r.as_euler('yxz', degrees=True))

In [10]:
query_image = '1LXtFkjw3qL_point0_query_0000'
db_image = '1LXtFkjw3qL_point0_database_0004'
path_to_hdf5_datasets = '/datasets/Habitat/1LXtFkjw3qL_point0/hdf5/'
hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
num_query = int(query_image.split('_')[-1])
num_db = int(db_image.split('_')[-1])

In [11]:
hdf5_filename_query

'1LXtFkjw3qL_point0.hdf5'

In [12]:
keypoints_file = h5py.File(str(outputs / feature_filename), 'r')
match_filename = 'feats-superpoint-n4096-r1024-nms4_matches-superglue_NetVLAD_query_top_40.h5'
match_file = h5py.File(str(outputs / match_filename), 'r')

In [13]:
keypoints_query = keypoints_file[query_image.rstrip('.png')]['keypoints'].__array__()
keypoints_db = keypoints_file[db_image.rstrip('.png')]['keypoints'].__array__()

In [14]:
images = {}
images['query'] = {}
images['db'] = {}
images['query']['points_3d'] = []
images['db']['points_3d'] = []
points_3D = {}
point_3D_id = 0
image_id = 1
depth_db = 10*hdf5_file_db['depth_base'][num_db]
cam_xyz_db = hdf5_file_db['gps_base'][num_db]
R_cam_db = hdf5_file_db['quat_base'][num_db]
# R_cam_db = [1,0,0,0]
# cam_xyz_db = [0,0,0]
for keypoint in keypoints_db:
    depth_keypoint = depth_db[int(keypoint[1]), int(keypoint[0])][0]
    point_3d_xyz = get_point_3d(keypoint[0], keypoint[1], depth_keypoint, fx, fy, cx, cy, cam_xyz_db, R_cam_db)
    if len(point_3d_xyz) != 1:
        if (math.isnan(depth_keypoint)) or (math.isinf(depth_keypoint)):
            images['db']['points_3d'].append(-1)
            continue
        else:
            images['db']['points_3d'].append(point_3D_id)
            points_3D[point_3D_id] = {}
            points_3D[point_3D_id]['xyz'] = point_3d_xyz
            if not('image_ids' in points_3D[point_3D_id].keys()):
                points_3D[point_3D_id]['image_ids'] = []
            if points_3D[point_3D_id]['image_ids'].count(image_id) == 0:
                points_3D[point_3D_id]['image_ids'] = [image_id]
            else:
                points_3D[point_3D_id]['image_ids'].append(image_id)
            point_3D_id += 1
    else:
        images['db']['points_3d'].append(-1)

image_id = 2
depth_query = 10*hdf5_file_query['depth'][num_query]
cam_xyz = hdf5_file_query['gps'][num_query]
R_cam = hdf5_file_query['quat'][num_query]
print(cam_xyz)
print(R_cam)
for keypoint in keypoints_query:
    depth_keypoint = depth_query[int(keypoint[1]), int(keypoint[0])][0]
    point_3d_xyz = get_point_3d(keypoint[0], keypoint[1], depth_keypoint, fx, fy, cx, cy, cam_xyz, R_cam)
    if len(point_3d_xyz) != 1:
        if (math.isnan(depth_keypoint)) or (math.isinf(depth_keypoint)):
            images['query']['points_3d'].append(-1)
        else:
            images['query']['points_3d'].append(point_3D_id)
        points_3D[point_3D_id] = {}
        points_3D[point_3D_id]['xyz'] = point_3d_xyz
        if not('image_ids' in points_3D[point_3D_id].keys()):
            points_3D[point_3D_id]['image_ids'] = []
        if points_3D[point_3D_id]['image_ids'].count(image_id) == 0:
            points_3D[point_3D_id]['image_ids'] = [image_id]
        else:
            points_3D[point_3D_id]['image_ids'].append(image_id)
        point_3D_id += 1
    else:
        images['query']['points_3d'].append(-1)
print(len(keypoints_query))

[ 0.11889225  0.2980008  -7.8185797 ]
[-0.09069052  0.          0.99587911  0.        ]
118


## G2O

In [15]:
class BundleAdjustment(g2o.SparseOptimizer):
    def __init__(self, ):
        super().__init__()

        # Higher confident (better than CHOLMOD, according to 
        # paper "3-D Mapping With an RGB-D Camera")
        solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
        solver = g2o.OptimizationAlgorithmLevenberg(solver)
        super().set_algorithm(solver)
        super().set_verbose(True)

        # Convergence Criterion
        terminate = g2o.SparseOptimizerTerminateAction()
        terminate.set_gain_threshold(1e-6)
        super().add_post_iteration_action(terminate)

        # Robust cost Function (Huber function) delta
        self.delta = np.sqrt(5.991)   
        self.aborted = False

    def optimize(self, max_iterations=10):
        super().initialize_optimization()
        super().optimize(max_iterations)
        try:
            return not self.aborted
        finally:
            self.aborted = False

    def add_pose(self, pose_id, pose, cam, fixed=False):
        sbacam = g2o.SBACam(
            pose.orientation(), pose.position())
        sbacam.set_cam(
            cam.fx, cam.fy, cam.cx, cam.cy, cam.baseline)

        v_se3 = g2o.VertexCam()
        v_se3.set_id(pose_id * 2)
        v_se3.set_estimate(sbacam)
        v_se3.set_fixed(fixed)
        super().add_vertex(v_se3) 

    def add_point(self, point_id, point, fixed=False, marginalized=True):
        v_p = g2o.VertexSBAPointXYZ()
        v_p.set_id(point_id * 2 + 1)
        v_p.set_marginalized(marginalized)
        v_p.set_estimate(point)
        v_p.set_fixed(fixed)
        super().add_vertex(v_p)

    def add_edge(self, id, point_id, pose_id, meas):
        if meas.is_stereo():
            edge = self.stereo_edge(meas.xyx)
        elif meas.is_left():
            edge = self.mono_edge(meas.xy)
        elif meas.is_right():
            edge = self.mono_edge_right(meas.xy)

        edge.set_id(id)
        edge.set_vertex(0, self.vertex(point_id * 2 + 1))
        edge.set_vertex(1, self.vertex(pose_id * 2))
        kernel = g2o.RobustKernelHuber(self.delta)
        edge.set_robust_kernel(kernel)
        super().add_edge(edge)

    def stereo_edge(self, projection, information=np.identity(3)):
        e = g2o.EdgeProjectP2SC()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def mono_edge(self, projection, 
            information=np.identity(2) * 0.5):
        e = g2o.EdgeProjectP2MC()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def mono_edge_right(self, projection, 
            information=np.identity(2) * 0.5):
        e = g2o.EdgeProjectP2MCRight()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def get_pose(self, id):
        return self.vertex(id * 2).estimate()

    def get_point(self, id):
        return self.vertex(id * 2 + 1).estimate()

    def abort(self):
        self.aborted = True

In [16]:
class Camera(object):
    def __init__(self, fx, fy, cx, cy, width, height, baseline):
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.baseline = baseline

        self.intrinsic = np.array([
            [fx, 0, cx], 
            [0, fy, cy], 
            [0, 0, 1]])

#         self.frustum_near = frustum_near
#         self.frustum_far = frustum_far

        self.width = width
        self.height = height
        
    def compute_right_camera_pose(self, pose):
        pos = pose * np.array([self.baseline, 0, 0])
        return g2o.Isometry3d(pose.orientation(), pos)

In [17]:
class GraphMeasurement(object):
    def __init__(self):
        self.keyframe = None
        self.mappoint = None

    @property
    def id(self):
        return (self.keyframe.id, self.mappoint.id)

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, rhs):
        return (isinstance(rhs, GraphMeasurement) and
            self.id == rhs.id)

In [18]:
class Measurement(GraphMeasurement):
    
    Source = Enum('Measurement.Source', ['TRIANGULATION', 'TRACKING', 'REFIND'])
    Type = Enum('Measurement.Type', ['STEREO', 'LEFT', 'RIGHT'])

    def __init__(self, type, source, keypoints):
        super().__init__()

        self.type = type
        self.source = source
        self.keypoints = keypoints
        self.view = None    # mappoint's position in current coordinates frame

        self.xy = np.array(self.keypoints[0].pt)
        if self.is_stereo():
            self.xyx = np.array([
                *keypoints[0].pt, keypoints[1].pt[0]])

        self.triangulation = (source == self.Source.TRIANGULATION)

    def get_keypoint(self, i=0):
        return self.keypoints[i]

    def get_keypoints(self):
        return self.keypoints

    def is_stereo(self):
        return self.type == Measurement.Type.STEREO
    def is_left(self):
        return self.type == Measurement.Type.LEFT
    def is_right(self):
        return self.type == Measurement.Type.RIGHT

    def from_triangulation(self):
        return self.triangulation
    def from_tracking(self):
        return self.source == Measurement.Source.TRACKING
    def from_refind(self):
        return self.source == Measurement.Source.REFIND

In [19]:
class GraphMapPoint(object):
    def __init__(self):
        self.id = None
        self.meas = dict()
        self._lock = Lock()

    def __hash__(self):
        return self.id

    def __eq__(self, rhs):
        return (isinstance(rhs, GraphMapPoint) and 
            self.id == rhs.id)
    def __lt__(self, rhs):
        return self.id < rhs.id
    def __le__(self, rhs):
        return self.id <= rhs.id

    def measurements(self):
        with self._lock:
            return self.meas.keys()

    def keyframes(self):
        with self._lock:
            return self.meas.values()

    def add_measurement(self, m):
        with self._lock:
            self.meas[m] = m.keyframe

    def remove_measurement(self, m):
        with self._lock:
            try:
                del self.meas[m]
            except KeyError:
                pass

In [20]:
class MapPoint(GraphMapPoint):
    _id = 0
    _id_lock = Lock()

    def __init__(self, position, covariance=np.identity(3) * 1e-4):
        super().__init__()

        with MapPoint._id_lock:
            self.id = MapPoint._id
            MapPoint._id += 1

        self.position = position
        self.covariance = covariance
        # self.owner = None

        self.count = defaultdict(int)

    def update_position(self, position):
        self.position = position
    def update_normal(self, normal):
        self.normal = normal
    def update_descriptor(self, descriptor):
        self.descriptor = descriptor
    def set_color(self, color):
        self.color = color

    def is_bad(self):
        with self._lock:
            status =  (
                self.count['meas'] == 0
                or (self.count['outlier'] > 20
                    and self.count['outlier'] > self.count['inlier'])
                or (self.count['proj'] > 20
                    and self.count['proj'] > self.count['meas'] * 10))
            return status

    def increase_outlier_count(self):
        with self._lock:
            self.count['outlier'] += 1
    def increase_inlier_count(self):
        with self._lock:
            self.count['inlier'] += 1
    def increase_projection_count(self):
        with self._lock:
            self.count['proj'] += 1
    def increase_measurement_count(self):
        with self._lock:
            self.count['meas'] += 1

In [21]:
fx, fy = 128., 128.
cx, cy = 128., 128.
width, height = 256, 256
fx_baseline = 300
baseline = fx_baseline/fx
cam = Camera(
    fx, fy, cx, cy, 
    width, height, 
    baseline)

### Optimization

In [22]:
pair = names_to_pair(query_image, db_image)
matches = match_file[pair]['matches0'].__array__()
valid = np.where(matches > -1)[0]
# valid - массив точек на query изображении, которые имеют сопоставление

In [32]:
np.sum(matches)

3578

In [23]:
print(query_image)
print(db_image)

1LXtFkjw3qL_point0_query_0000
1LXtFkjw3qL_point0_database_0004


In [24]:
# For Query image

data = {
    'points_db'   : [],
    'points_g2o'  : [],
    'points_query': []
}

optimizer = BundleAdjustment()
optimizer.clear()
pose_44 = np.eye(4)
R_cam = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])

r = R.from_quat([hdf5_file_db['quat_base'][num_db][1], hdf5_file_db['quat_base'][num_db][2], hdf5_file_db['quat_base'][num_db][3], hdf5_file_db['quat_base'][num_db][0]])
matrix = r.as_matrix()

pose_44[:3,:3] = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
pose_44[:3,3] = hdf5_file_db['gps_base'][num_db]
pose = g2o.Isometry3d(pose_44)
optimizer.add_pose(0, pose, cam, fixed=False)
measurements = []
for i, keypoint_xy_query in enumerate(keypoints_query):
    # Если кипоинт не имеет сопоставления, то пропускаем его
    if list(valid).count(i) == 0:
        continue
    # Получим индекс соответствующего кипоинта на database изображении
    kp_idx_db = matches[i]
    # Получим индекс point 3D, соответствующий индексу ключевой точки kp_idx_db
    point_3D_id_db = images['db']['points_3d'][kp_idx_db]
    point_3D_id_query = images['query']['points_3d'][i]
    
    point_xyz_db    = points_3D[point_3D_id_db]['xyz']
    point_xyz_query = points_3D[point_3D_id_query]['xyz']
    
    data['points_db'].append(point_xyz_db.tolist())
    data['points_query'].append(point_xyz_query.tolist())
    
    x_right = keypoint_xy_query[0] - fx_baseline/(depth_query[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0])
    kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
    kp_left.pt  = (int(keypoint_xy_query[0]), int(keypoint_xy_query[1]))
    kp_right.pt = (int(x_right), int(keypoint_xy_query[1]))
    meas = Measurement(Measurement.Type.STEREO,
                       Measurement.Source.TRIANGULATION,
                       [kp_left, kp_right])
    mappoint = MapPoint(point_xyz_db)
    meas.mappoint = mappoint
    measurements.append(meas)

In [25]:
# # For database image
# optimizer = BundleAdjustment()
# optimizer.clear()
# pose_44 = np.eye(4)
# R_cam = hdf5_file_db['quat_base'][num_db]
# r = R.from_quat([R_cam[1], R_cam[2], R_cam[3], R_cam[0]])
# pose_44[:3,:3] = r.as_matrix()
# pose_44[:3,3] = hdf5_file_db['gps_base'][num_db]
# pose = g2o.Isometry3d(pose_44)
# optimizer.add_pose(0, pose, cam, fixed=False)
# measurements = []
# for i, keypoint in enumerate(keypoints_db):
#     kp_idx_db = i
#     # Получим индекс point 3D, соответствующий индексу ключевой точки kp_idx_db
#     point_3D_id = images['db']['points_3d'][i]
#     point = points_3D[point_3D_id]['xyz']
    
#     x_right = keypoint[0] - fx_baseline/(depth_db[int(keypoint[1]), int(keypoint[0])][0])
#     kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
#     kp_left.pt  = (keypoint[0], keypoint[1])
#     kp_right.pt = (x_right, keypoint[1])
#     meas = Measurement(Measurement.Type.STEREO,
#                        Measurement.Source.TRIANGULATION,
#                        [kp_left, kp_right])
#     mappoint = MapPoint(point)
#     meas.mappoint = mappoint
#     measurements.append(meas)

In [26]:
for i, m in enumerate(measurements):
    optimizer.add_point(i, m.mappoint.position, fixed=True)
    optimizer.add_edge(0, i, 0, m)

In [27]:
optimizer.optimize(10)
result = optimizer.get_pose(0)
pose_estimated = result.matrix()

r = R.from_matrix(result.matrix()[:3,:3])
yaw_pitch_roll = r.as_euler('yxz', degrees=True)

print('g2o result:')
print(result.position())
print(yaw_pitch_roll)
print()
cam_xyz = hdf5_file_query['gps'][num_query]
pose_query = np.eye(4)
pose_query[:3,3] = cam_xyz
pose_query[:3,:3] = quaternion_to_rotation_matrix(hdf5_file_query['quat'][num_query])
print('query gt:')
print(cam_xyz)
r = R.from_matrix(pose_query[:3,:3])
yaw_pitch_roll = r.as_euler('yxz', degrees=True)
print(yaw_pitch_roll)
print()
cam_xyz_db = hdf5_file_db['gps_base'][num_db]
r = R.from_matrix(quaternion_to_rotation_matrix(hdf5_file_query['quat_base'][num_db]))
yaw_pitch_roll = r.as_euler('yxz', degrees=True)

# pose_estimated = np.eye(4)
# pose_estimated[:3,3] = cam_xyz_db
# pose_estimated[:3,:3] = r.as_matrix()

print('db pose:')
print(cam_xyz_db)
print(yaw_pitch_roll)

error_pose = np.linalg.inv(pose_estimated) @ pose_query
dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
r = R.from_matrix(error_pose[:3, :3])
rotvec = r.as_rotvec()
angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
angle_error = abs(90 - abs(angle_error-90))
print()
print('distance error - {}'.format(dist_error))
print('angle error - {}'.format(angle_error))
print('pose:')
print(result.matrix())
data['cam_pose_g2o'] = result.matrix().tolist()

g2o result:
[ 0.13260147  0.304025   -7.92440136]
[-1.06120887e+01 -8.48145196e-02 -1.79837403e+02]

query gt:
[ 0.11889225  0.2980008  -7.8185797 ]
[-10.40666736   0.         180.        ]

db pose:
[-0.10379431  0.26535606 -7.9203405 ]
[-21.10587093   0.         180.        ]

distance error - 0.10687592142804575
angle error - 0.27546089226920856
pose:
[[-9.82891784e-01  2.83785101e-03  1.84162121e-01  1.32601467e-01]
 [-3.06192455e-03 -9.99994878e-01 -9.32352448e-04  3.04025000e-01]
 [ 1.84158532e-01 -1.48029208e-03  9.82895439e-01 -7.92440136e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [29]:
len(keypoints_query)

106

In [30]:
for i, keypoint_xy_query in enumerate(keypoints_query):
    # Если кипоинт не имеет сопоставления, то пропускаем его
    if list(valid).count(i) == 0:
        continue
    depth_keypoint = depth_query[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
    point_3d_xyz_g2o = get_point_3d(keypoint_xy_query[0], keypoint_xy_query[1], depth_keypoint, fx, fy, cx, cy, result.matrix()[:3,3], result.matrix()[:3,:3])
    point_3d_xyz_query = get_point_3d(keypoint_xy_query[0], keypoint_xy_query[1], depth_keypoint, fx, fy, cx, cy, pose_query[:3,3], pose_query[:3,:3])
    if len(point_3d_xyz) != 1:
        if (math.isnan(depth_keypoint)) or (math.isinf(depth_keypoint)):
            continue
        else:
#             data['points_query'].append(point_3d_xyz_query.tolist())
            data['points_g2o'].append(point_3d_xyz_g2o.tolist())

In [31]:
with open('/home/g2o_data.json', 'w') as outfile:
    json.dump(data, outfile)

In [32]:
len(keypoints_db)

59

# Ransac

In [ ]:
pair = names_to_pair(query_image, db_image)
# if not(pair in match_file.keys()):
#     pair = names_to_pair(db_image, query_image)
# matches - список индексов ключевых точек на query изображении в порядке сопоставления с кипоинтами db
# matches = match_file[pair]['matches0'].__array__()
matches = np.arange(len(keypoints_db))
# valid - список индексов ключевых точек на query изображении, которые имеют сопоставление
valid = np.where(matches > -1)[0]
# matched_keypoints_query, matched_keypoints_db = keypoints_query[valid], keypoints_db[matches[valid]]
# num_matches = len(matched_keypoints_query)
points3D_ids = np.array(images['db']['points_3d'])
# matched_keypoints_query = keypoints_db
valid = valid[points3D_ids[matches[valid]] != -1]
kp_idx_to_3D = defaultdict(list)
kp_idx_to_3D_to_db = defaultdict(lambda: defaultdict(list))
i = 0
# Пробегаемся по индексам ключевых точек на изображении query, которые имеют сопоставление с db
# for idx in valid:
#     # берем индекс (idx) ключевой точки на изображении query;
#     # затем берем индекс кипоинта на изображении db, соответствующий этому кипоинту на query -  matches[idx]
#     # затем получаем индекс 3D ключевой точки по ключу индекса ключевой точки на db изображении
#     id_3D = points3D_ids[matches[idx]]
#     kp_idx_to_3D_to_db[idx][id_3D].append(i)
#     # avoid duplicate observations
#     if id_3D not in kp_idx_to_3D[idx]:
#         kp_idx_to_3D[idx].append(id_3D)
idxs = list(kp_idx_to_3D.keys())
mkp_idxs = [i for i in idxs for _ in kp_idx_to_3D[i]]
mkpq = keypoints_query[mkp_idxs]
# mkpq += 0.5  # COLMAP coordinates

mp3d_ids = [j for i in idxs for j in kp_idx_to_3D[i]]

mp3d = [list(points_3D[num_keypoint_in_db]['xyz']) for num_keypoint_in_db in valid if \
             images['db']['points_3d'][num_keypoint_in_db] != -1]
# mp3d = [points_3D[j]['xyz'] for j in mp3d_ids]
# mp3d = np.array(mp3d).reshape(-1, 3)

# mostly for logging and post-processing
# mkp_to_3D_to_db = [(j, kp_idx_to_3D_to_db[i][j])
#                    for i in idxs for j in kp_idx_to_3D[i]]

# camera_model, width, height, params = qinfo
cfg = {
       'model': 'PINHOLE',
       'width': int(width),
       'height': int(height),
       'params': np.array([fx, fy, cx, cy]),
}
# matched_keypoints_query += 0.5
keypoints_db += 0.5
ret = pycolmap.absolute_pose_estimation(keypoints_db, mp3d, cfg, 24)
print(ret['success'])
print(ret['tvec'])
print(yaw_pitch_roll_from_quaternion(list(ret['qvec']), w_first=True))

In [ ]:
ret['qvec']

In [ ]:
# All 6 images of base point. HFOV=90 degrees. TURN_ANGLE=60 degrees
num_db = 4
rgb_base = hdf5_file_db['rgb_base'][num_db]
rgb = hdf5_file_query['rgb'][num_query]
gps_base = hdf5_file_db['gps_base']
gps = hdf5_file_query['gps']
f = plt.figure(figsize=(10,10))
ax1 = f.add_subplot(121)
ax1.set_title('query')
ax2 = f.add_subplot(122)
ax2.imshow(rgb_base)
ax1.imshow(rgb)
ax2.set_title('database')

db_quat_wxyz = hdf5_file_db['quat_base'][num_db]
r = R.from_quat([db_quat_wxyz[1], db_quat_wxyz[2], db_quat_wxyz[3], db_quat_wxyz[0]])
euler_x_db, euler_y_db, euler_z_db = r.as_euler('xyz', degrees=True)
print('database pose:')
print('    translation: {}'.format(gps_base[num_db]))
print('    yaw, pitch, roll: {:.2f}, {:.2f}, {:.2f}'.format(euler_x_db, euler_y_db, euler_z_db))

r = R.from_quat([ret['qvec'][1], ret['qvec'][2], ret['qvec'][3], ret['qvec'][0]])
euler_x_pred, euler_y_pred, euler_z_pred = r.as_euler('xyz', degrees=True)
print()
print('query pose calculated:')
print('    translation: {}'.format(ret['tvec']))
print('    yaw, pitch, roll: {:.2f}, {:.2f}, {:.2f}'.format(euler_x_pred, euler_y_pred, euler_z_pred))

gt_quat_wxyz = hdf5_file_query['quat'][num_query]
r = R.from_quat([gt_quat_wxyz[1], gt_quat_wxyz[2], gt_quat_wxyz[3], gt_quat_wxyz[0]])
euler_x_gt, euler_y_gt, euler_z_gt = r.as_euler('xyz', degrees=True)
print()
print('query pose gt:')
print('    translation: {}'.format(gps[num_query]))
print('    yaw, pitch, roll: {:.2f}, {:.2f}, {:.2f}'.format(euler_x_gt, euler_y_gt, euler_z_gt))

In [ ]:
root_dir_images = images_path
first_image = Path(db_image)
second_image = Path(query_image)
keypoints_filename = Path(feature_file)
visualization.visualize_matches(str(outputs / keypoints_filename), str(outputs / match_filename), root_dir_images, os.path.join(root_dir_images, second_image), os.path.join(root_dir_images, first_image), dpi=300)

# Visualization of keypoints

In [ ]:
keypoints_new = []
for i, keypoint in enumerate(keypoints_db):
    kp_idx_db = i
    # Получим индекс point 3D, соответствующий индексу ключевой точки kp_idx_db
    point_3D_id = images['db']['points_3d'][i]
    point = points_3D[point_3D_id]['xyz']
    x_right = keypoint[0] - fx_baseline/(depth_db[int(keypoint[1]), int(keypoint[0])][0])
    kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
    kp_left.pt  = (keypoint[0], keypoint[1])
    kp_right.pt = (x_right, keypoint[1])
    meas = Measurement(Measurement.Type.STEREO,
                       Measurement.Source.TRIANGULATION,
                       [kp_left, kp_right])
    mappoint = MapPoint(point)
    meas.mappoint = mappoint
    measurements.append(meas)

In [ ]:
cam_pose = result.matrix()
rotation_matrix = cam_pose[:3,:3]
rotation_matrix = quaternion_to_rotation_matrix(R_cam_db)
# cam_pose = np.eye(4)
# cam_pose[:3,:3] = rotation_matrix
# cam_pose[:3,3] = cam_xyz_db

In [ ]:
list_of_kps = []
for point_3D_id, point_3D in points_3D.items():
    if point_3D['image_ids'] == [2]:
        continue
    xyz = point_3D['xyz'] - cam_pose[:3, 3]
    xyz = np.matmul(np.linalg.inv(rotation_matrix), xyz)
    x = fx * xyz[0] / xyz[2] + cx
    y = fy * xyz[1] / xyz[2] + cy
    list_of_kps.append([int(x),int(y)])
array_of_kps = np.array([np.array(xi) for xi in list_of_kps])

In [ ]:
from hloc.utils.viz import plot_images, plot_keypoints, plot_matches, cm_RdGn

In [ ]:
def read_image(path):
    appendix = str(path).split('_')
    directory = str(path)[:str(path).rfind('/')]
    subdir = appendix[-4][appendix[-4].rfind('/'):]+'_'+appendix[-3]
    subdir = subdir[1:]
    filename = str(path).split('/')[-1] + '.png'
#     path = directory + '/' + subdir + '/' +  filename
    path = directory + '/' +  filename
    path = Path(path)
    assert path.exists(), path
    image = cv2.imread(str(path))
    if len(image.shape) == 3:
        image = image[:, :, ::-1]
    return image

In [ ]:
image = read_image(os.path.join(images_path, db_image))

In [ ]:
plot_images([image], dpi=300)
plot_keypoints([array_of_kps], colors='lime', ps=4)

In [ ]:
plot_images([image], dpi=300)
plot_keypoints([keypoints_db], colors='lime', ps=4)

In [ ]:
pwd

# Getting poses for query images

In [33]:
def quaternion_to_rotation_matrix(quaternion_wxyz):
    r = R.from_quat([quaternion_wxyz[1], quaternion_wxyz[2], quaternion_wxyz[3], quaternion_wxyz[0]])
    matrix = r.as_matrix()
    matrix[:3,2] = -matrix[:3,2]
    matrix[:3,1] = -matrix[:3,1]
    return matrix

In [34]:
def get_point_3d(x, y, depth, fx, fy, cx, cy, cam_center_world, R_world_to_cam, w_in_quat_first = True):
    if depth <= 0:
        return 0
    new_x = (x - cx)*depth/fx
    new_y = (y - cy)*depth/fy
    new_z = depth
    coord_3D_world_to_cam = np.array([new_x, new_y, new_z], float)
    if len(R_world_to_cam) == 4:
        if w_in_quat_first:
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
        else:
            R_world_to_cam = [R_world_to_cam[3], R_world_to_cam[0], R_world_to_cam[1], R_world_to_cam[2]]
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
    coord_3D_cam_to_world = np.matmul(matrix, coord_3D_world_to_cam) + cam_center_world
    return coord_3D_cam_to_world

In [35]:
def yaw_pitch_roll_from_quaternion(quaternion, w_first=False):
    if not(w_first):
        r = R.from_quat(quaternion)
    else:
        r = R.from_quat([quaternion[1], quaternion[2], quaternion[3] ,quaternion[0]])
    return r.as_euler('yxz', degrees=True)

In [36]:
# loc_pairs = '/datasets/Habitat/NetVLAD.txt'
retrievals = parse_retrieval(loc_pairs)

In [37]:
path_to_hdf5_datasets = '/datasets/Habitat/1LXtFkjw3qL_point0/hdf5/'
keypoints_file = h5py.File(str(outputs / feature_filename), 'r')
match_filename = 'feats-superpoint-n4096-r1024-nms4_matches-superglue_NetVLAD_query_top_40.h5'
match_file = h5py.File(str(outputs / match_filename), 'r')

In [38]:
fx, fy = 128., 128.
cx, cy = 128., 128.
width, height = 256, 256
fx_baseline = 300
baseline = fx_baseline/fx

cam = Camera(
    fx, fy, cx, cy, 
    width, height, 
    baseline)

top_num_db_image = 1

In [47]:
# G2O

data = {
    'query'    : {},
    'database' : {},
    'g2o'      : {}
}

optimizer = BundleAdjustment()
results = {}
metrics = {
    "(5m, 20°)"   : 0,
    "(1m, 10°)"   : 0,
    "(0.5m, 5°)"  : 0,
    "(0.25m, 2°)" : 0
}
max_angle_error = 0
for query_image, db_images in tqdm(retrievals.items()):
#     if query_image != '1LXtFkjw3qL_point5_query_0000':
#         continue
    measurements = []
    optimizer.clear()
    hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
    hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
    
    num_query = int(query_image.split('_')[-1])
    
    depth_query = 10*hdf5_file_query['depth'][num_query]
    
    # keypoints_query - массив из пар координта (x,y), которые являются кипоинтами на query image
    keypoints_query = keypoints_file[query_image]['keypoints'].__array__()
    
    # all_matched_keypoints_ids - массив из true/false, говорящий о том, имеет ли совпадение кипоинт с соответствующим индексом
    all_matched_keypoints_ids = np.full((len(keypoints_query)), False)
    
    for num in range(len(db_images)):
        if num >= top_num_db_image:
            break
        db_image = db_images[num]
#         if db_image != '1LXtFkjw3qL_point5_database_0004':
#             continue
        hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
        hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
        
        num_db = int(db_image.split('_')[-1])
        
        depth_db = 10*hdf5_file_db['depth_base'][num_db]
        
        translation = hdf5_file_db['gps_base'][num_db]
        orientation = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
        pose_44 = np.eye(4)
        pose_44[:3,:3] = orientation
        pose_44[:3,3] = translation
        data['database']['cam_pose'] = pose_44
        pose = g2o.Isometry3d(pose_44)
        optimizer.add_pose(0, pose, cam, fixed=False)
        
        keypoints_db = keypoints_file[db_image]['keypoints'].__array__()
        
        # matches_db_keypoints_ids - массив из индексов кипоинтов на database image, которые совпадают с кипоинтом, индекс которого равен положению в массиве
        matches_db_keypoints_ids = match_file['_'.join([query_image,db_image])]['matches0'].__array__()
        
        # matched_keypoints_query_ids - массив из индексов кипоинтов на query image, которые имеют сопоставление
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        
        # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
        for keypoint_id_query in matched_keypoints_query_ids:
            if all_matched_keypoints_ids[keypoint_id_query] == False:
                
                # затем получаем индекс сопоставленного кипоинта на database image
                keypoint_id_database = matches_db_keypoints_ids[keypoint_id_query]
                
                # получаем координаты x,y для сопоставленных кипоинтов на query и database
                keypoint_xy_query = keypoints_query[keypoint_id_query]
                keypoint_xy_db = keypoints_db[keypoint_id_database]
                
                # получаем глубины данных кипоинтов
                depth_keypoint_query = depth_query[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
                depth_keypoint_db = depth_db[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]
                
                # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
                if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
                    continue
                    
                # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
                x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query
                
                # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
                kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
                kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
                kp_right.pt = (x_right, keypoint_xy_query[1])
                meas = Measurement(Measurement.Type.STEREO,
                       Measurement.Source.TRIANGULATION,
                       [kp_left, kp_right])
                
                #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
                # с кипоинтом на query image
                # Получаем 3D координату ключевой точки на database image
                global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
                                                      fx, fy, cx, cy, \
                                                      translation, hdf5_file_db['quat_base'][num_db])
                mappoint = MapPoint(global_xyz_of_keypoint)
                # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
                # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
                meas.mappoint = mappoint
                measurements.append(meas)
                
                # Помечаем этот кипоинт на query image тем, что в дальнейшем его не нужно рассматривать - его уже
                # добавили в измерения
                all_matched_keypoints_ids[keypoint_id_query] = True
    for i, m in enumerate(measurements):
        optimizer.add_point(i, m.mappoint.position, fixed=True)
        optimizer.add_edge(0, i, 0, m)
    optimizer.optimize(10)
    result = optimizer.get_pose(0)
    data['g2o']['cam_pose'] = result
    
    pose_estimated = np.eye(4)
    pose_query = np.eye(4)
    
#     r = R.from_quat([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
    
#     # Now get pose from g2o optimization 
    matrix = result.matrix()[:3,:3]
    pose_estimated[:3, :3] = matrix
    pose_estimated[:3, 3] = result.position()
#     print("Estimated translation:")
#     print(result.position())
#     print(matrix)

    # Now get pose from NetVLAD
    pose_estimated[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
    pose_estimated[:3, 3] = hdf5_file_db['gps_base'][num_db]
    
#     orientation_quat_xyzw = [hdf5_file_query['quat'][num_query][1], hdf5_file_query['quat'][num_query][2],\
#                             hdf5_file_query['quat'][num_query][3], hdf5_file_query['quat'][num_query][0]]
#     r = R.from_quat(orientation_quat_xyzw)
#     matrix = r.as_matrix()
    pose_query[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_query['quat'][num_query])
    pose_query[:3, 3] = hdf5_file_query['gps'][num_query]
    data['query']['cam_pose'] = pose_query 
#     print("GT translation:")
#     print(hdf5_file_query['gps'][num_query])
    error_pose = np.linalg.inv(pose_estimated) @ pose_query
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(90 - abs(angle_error-90))
#     if angle_error > max_angle_error:
#         max_angle_error = angle_error
#         print(query_image)
#         print(db_image)
#         print(max_angle_error)
#         print("len of measurements = {}".format(len(measurements)))
#         print()
#         print()
#     print("angle error = {.f}", angle_error)
#     print("dist error = {.f}", dist_error)
#         print(db_image)
#         print(query_image)
#         print()

#     results[query_image] = {
#         'xyz': list(map(float, result.position())),
#         'quaternion_xyzw': list(map(float,r.as_quat()))
#     }
#     if angle_error < 20 and dist_error < 5:
#         metrics["(5m, 20°)"] += 1
#     if angle_error < 10 and dist_error < 1:
#         metrics["(1m, 10°)"] += 1
#     if angle_error < 5 and dist_error < 0.5:
#         metrics["(0.5m, 5°)"] += 1
#     if angle_error < 2 and dist_error < 0.25:
#         metrics["(0.25m, 2°)"] += 1
    if dist_error < 5:
        metrics["(5m, 20°)"] += 1
    if dist_error < 1:
        metrics["(1m, 10°)"] += 1
    if dist_error < 0.5:
        metrics["(0.5m, 5°)"] += 1
    if dist_error < 0.25:
        metrics["(0.25m, 2°)"] += 1
#     break
for key in metrics.keys():
    metrics[key] = metrics[key] / 950

100%|██████████| 950/950 [00:26<00:00, 35.75it/s]


In [48]:
metrics

{'(5m, 20°)': 0.9705263157894737,
 '(1m, 10°)': 0.9189473684210526,
 '(0.5m, 5°)': 0.9189473684210526,
 '(0.25m, 2°)': 0.5021052631578947}

In [ ]:
metrics

с использованием магической строчки

In [ ]:
metrics

In [ ]:
metrics

In [ ]:
# RANSAC

results = {}
metrics = {
    "(5m, 20°)": 0,
    "(1m, 10°)": 0,
    "(0.5m, 5°)": 0,
    "(0.25m, 2°)": 0
}
for query_image, db_images in tqdm(retrievals.items()):
    keypoints_2D = []
    keypoints_3D = []
    hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
    hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
    
    num_query = int(query_image.split('_')[-1])
    
    keypoints_query = keypoints_file[query_image]['keypoints'].__array__()
    all_matched_keypoints_ids = np.full((len(keypoints_query)), False)
    
    for num in range(len(db_images)):
        if num >= top_num_db_image:
            break
        db_image = db_images[num]
        hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
        hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
        
        num_db = int(db_image.split('_')[-1])
        
        depth_db = 10*hdf5_file_db['depth_base'][num_db]
        
        translation = hdf5_file_db['gps_base'][num_db]
        orientation_matrix_44 = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
        pose_44 = np.eye(4)
        pose_44[:3,:3] = orientation_matrix_44
        pose_44[:3,3] = translation
        
        matches_db_keypoints_ids = match_file['_'.join([query_image,db_image])]['matches0'].__array__()
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        for keypoint_id_query in matched_keypoints_query_ids:
            if all_matched_keypoints_ids[keypoint_id_query] == False:
                keypoint = keypoints_query[keypoint_id_query]
                depth_keypoint = depth_db[int(keypoint[1]), int(keypoint[0])][0]
                if depth_keypoint <= 0:
                    continue
                global_xyz_of_keypoint = get_point_3d(keypoint[0], keypoint[1], depth_keypoint, \
                                                      fx, fy, cx, cy, \
                                                      translation, hdf5_file_db['quat_base'][num_db])
                keypoint += 0.5
                keypoints_2D.append(list(keypoint))
                keypoints_3D.append(list(global_xyz_of_keypoint))
                all_matched_keypoints_ids[keypoint_id_query] = True           
#     ret = pycolmap.absolute_pose_estimation(keypoints_2D, keypoints_3D, cfg, 12)
#     print(ret['success'])
#     print(ret['tvec'])
#     print(yaw_pitch_roll_from_quaternion(list(ret['qvec']), w_first=True))
#     break
#     r = R.from_matrix(result.matrix()[:3,:3])
#     yaw_pitch_roll = r.as_euler('yxz', degrees=True)
    orientation_quat_wxyz = hdf5_file_db['quat_base'][num_db]
#     results[query_image] = {
#         'xyz': list(map(float,translation)),
#         'quaternion_xyzw': list([float(orientation[1]), float(orientation[2]), float(orientation[3]), float(orientation[0])])
#     }
#     r = R.from_quat(list([orientation[1], orientation[2], orientation[3], orientation[0]]))
#     yaw_pitch_roll = r.as_euler('yxz', degrees=True)
#     print(query_image)
#     print("NetVLAD")
#     print([orientation[1], orientation[2], orientation[3], orientation[0]])
#     print(translation)
#     print(yaw_pitch_roll)
#     print("query")

    pose_estimated = np.eye(4)
    pose_query = np.eye(4)
    
    r = R.from_quat([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
#     print([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
    pose_estimated[:3, :3] = r.as_matrix()
    pose_estimated[:3, 3] = translation
    
    orientation_quat_xyzw = [hdf5_file_query['quat'][num_query][1], hdf5_file_query['quat'][num_query][2],\
                            hdf5_file_query['quat'][num_query][3], hdf5_file_query['quat'][num_query][0]]
    r = R.from_quat(orientation_quat_xyzw)
    pose_query[:3, :3] = r.as_matrix()
    pose_query[:3, 3] = hdf5_file_query['gps'][num_query]
    error_pose = np.linalg.inv(pose_estimated) @ pose_query
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(90 - abs(angle_error-90))
    
    if dist_error < 5: #and angle_error < 20 and dist_error < 5:
        metrics["(5m, 20°)"] += 1
    if dist_error < 1: #and angle_error < 10:
        metrics["(1m, 10°)"] += 1
    if dist_error < 0.5: #and angle_error < 5:
        metrics["(0.5m, 5°)"] += 1
    if dist_error < 0.25: #and angle_error < 2:
        metrics["(0.25m, 2°)"] += 1
for key in metrics.keys():
    metrics[key] = metrics[key] / 950

In [ ]:
metrics

In [ ]:
with open(str(outputs / 'localization_results.json'), 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
num_query = int(query_image.split('_')[-1])
translation_query = hdf5_file_query['gps'][num_query]
orientation = quaternion_to_rotation_matrix(hdf5_file_query['quat'][num_query])
r = R.from_matrix(orientation)
yaw_pitch_roll = r.as_euler('yxz', degrees=True)
print(translation_query)
print(yaw_pitch_roll)

In [ ]:
len(results.keys())